# HearHere
## Measure local reputation of global business

### 1. Choose company and extract locations

#### 1.1 'BBVA' offices extractor

In [1]:
import pandas as pd
import numpy as np
import json

from config import data_path

In [2]:
#Setup functions requests and path
update_bbva_offices = False
bbva_offices_path = data_path + 'bbva_offices.csv'

#Load data
if update_bbva_offices: 
    
    #Import BBVA offices scraper
    from scrapers import bbva_extract_offices
    
    #Export to .csv
    bbva_offices = bbva_extract_offices()
    bbva_offices.to_csv(bbva_offices_path, index=False)
    
else:
    bbva_offices = pd.read_csv(bbva_offices_path)
    
#Drop unneeded columns
bbva_offices = bbva_offices.drop(columns=['office_url', 'municipality_url', 'province_url'])

#Drop empty rows
bbva_offices.replace('', np.nan, inplace=True)
bbva_offices = bbva_offices.dropna()
    
bbva_offices.head()

,office_number,office_direction,municipality_name,province_name
0,92.0,"Adra - Natalio Rivas, 23, C.P:4770, Adra, Alm...",Adra,Almería
1,5764.0,"Aguadulce - Av. Carlos Iii Esq. C/lima, C.P:47...",Aguadulce,Almería
2,5461.0,"Almeria - Empresas - Paseo Almeria, 41, C.P:40...",Almeria,Almería
3,1809.0,"Almeria - Ps. Almeria 41 - Ps. Almeria, 41, C....",Almeria,Almería
4,6828.0,Almeria - Edf Oliveros - Canonigo Molina Alons...,Almeria,Almería


#### 1.2 Improve 'office_direction' data

In [3]:
#Remove the first municipality appearance in 'office_direction'
bbva_offices['office_direction'] = bbva_offices['office_direction'].replace({r'^.+? - ': ''}, regex=True)
bbva_offices.head()

,office_number,office_direction,municipality_name,province_name
0,92.0,"Natalio Rivas, 23, C.P:4770, Adra, Almería",Adra,Almería
1,5764.0,"Av. Carlos Iii Esq. C/lima, C.P:4720, Aguadulc...",Aguadulce,Almería
2,5461.0,"Empresas - Paseo Almeria, 41, C.P:4001, Almeri...",Almeria,Almería
3,1809.0,"Ps. Almeria 41 - Ps. Almeria, 41, C.P:4001, Al...",Almeria,Almería
4,6828.0,"Edf Oliveros - Canonigo Molina Alonso, 6, C.P:...",Almeria,Almería


In [4]:
#Extract postal code
bbva_offices['postal_code'] = bbva_offices['office_direction'].str.extract(pat = 'C\.P:(\d*)', expand=True)
bbva_offices.head()

,office_number,office_direction,municipality_name,province_name,postal_code
0,92.0,"Natalio Rivas, 23, C.P:4770, Adra, Almería",Adra,Almería,4770
1,5764.0,"Av. Carlos Iii Esq. C/lima, C.P:4720, Aguadulc...",Aguadulce,Almería,4720
2,5461.0,"Empresas - Paseo Almeria, 41, C.P:4001, Almeri...",Almeria,Almería,4001
3,1809.0,"Ps. Almeria 41 - Ps. Almeria, 41, C.P:4001, Al...",Almeria,Almería,4001
4,6828.0,"Edf Oliveros - Canonigo Molina Alonso, 6, C.P:...",Almeria,Almería,4004


In [5]:
#Remove postal code, province and municipality from 'office_direction'
bbva_offices['office_direction'] = bbva_offices['office_direction'].replace({r', C\.P:.*': ''}, regex=True)
bbva_offices.head()

,office_number,office_direction,municipality_name,province_name,postal_code
0,92.0,"Natalio Rivas, 23",Adra,Almería,4770
1,5764.0,Av. Carlos Iii Esq. C/lima,Aguadulce,Almería,4720
2,5461.0,"Empresas - Paseo Almeria, 41",Almeria,Almería,4001
3,1809.0,"Ps. Almeria 41 - Ps. Almeria, 41",Almeria,Almería,4001
4,6828.0,"Edf Oliveros - Canonigo Molina Alonso, 6",Almeria,Almería,4004


#### 2. API Configuration

In [6]:
#Import Google Maps API key and setup the library
from config import geocoding_api_key
if geocoding_api_key == '':
    from api_keys import geocoding_api_key

import googlemaps    
gmaps = googlemaps.Client(key=geocoding_api_key)

In [12]:
import re

#Import Google Maps API key and setup the library
from config import geocoding_api_key
if geocoding_api_key == '':
    from api_keys import geocoding_api_key

import googlemaps    
gmaps = googlemaps.Client(key=geocoding_api_key)
    

def extract_geocode(address):

    print(address, type(address))
    
    #Build the url
    address = 'BBVA ' + address
    
    #Change any symbols substring with '+'
    address = re.sub(r'[^0-9a-zA-Z]+', '+', address)
    
    #Call the API and access to the first result if there are more    
    geocode_result = gmaps.geocode(address)
    
    try:
        geocode_result = geocode_result[0]

        #Define variables
        formatted_address = geocode_result['formatted_address']
        lat = geocode_result['geometry']['location']['lat']
        lng = geocode_result['geometry']['location']['lng']
        place_id = geocode_result['place_id']
                
        #Return requested data as a list
        return ([formatted_address, lat, lng, place_id])
    
    #No results
    except:
        return ('')

bbva_offices['geo_extraction'] = bbva_offices.apply(lambda row: extract_geocode(row['office_direction']), axis=1)
bbva_offices.head()

Natalio Rivas, 23 <class 'str'>
BBVA+Natalio+Rivas+23 <class 'str'>
Av. Carlos Iii Esq. C/lima <class 'str'>
BBVA+Av+Carlos+Iii+Esq+C+lima <class 'str'>
['Av. de Carlos III, 04720 Aguadulce, Almería, Spain', 36.8148931, -2.5713492, 'ChIJgYk3fQlxcA0Rg3xc24eoiwE']
Empresas - Paseo Almeria, 41 <class 'str'>
BBVA+Empresas+Paseo+Almeria+41 <class 'str'>
['Paseo de Almería, 41, 04001 Almería, Spain', 36.8384804, -2.4631383, 'ChIJe_rkcgF2cA0R7LhCBawLpCc']
Ps. Almeria 41 - Ps. Almeria, 41 <class 'str'>
BBVA+Ps+Almeria+41+Ps+Almeria+41 <class 'str'>
['Paseo de Almería, 41, 04001 Almería, Spain', 36.8384804, -2.4631383, 'ChIJe_rkcgF2cA0R7LhCBawLpCc']
Edf Oliveros - Canonigo Molina Alonso, 6 <class 'str'>
BBVA+Edf+Oliveros+Canonigo+Molina+Alonso+6 <class 'str'>
['Calle Canónigo Molina Alonso, 6, 04004 Almería, Spain', 36.8346925, -2.4617985, 'ChIJPaRhYAJ2cA0RllqXwOJkh08']
Nueva Andalucia - Arcipreste De Hita, 2 Esq. Santiago <class 'str'>
BBVA+Nueva+Andalucia+Arcipreste+De+Hita+2+Esq+Santiago <cl

['Calle Cartuja, S/N, 11401 Jerez de la Frontera, Cádiz, Spain', 36.6807712, -6.1287886, 'ChIJv9Fsqu3GDQ0RqhFlPIvm52Y']
Naviarcos - Lola Flores,18 Edif.expositor L.21a <class 'str'>
BBVA+Naviarcos+Lola+Flores+18+Edif+expositor+L+21a <class 'str'>
['Av. Lola Flores, 18, 11405 Jerez de la Frontera, Cádiz, Spain', 36.6908773, -6.1122591, 'ChIJ69NjWt7GDQ0RB5m3TsirGGg']
Av.arcos De La Fra - Av.arcos De La Frontera, Bl.6 Mzna.6 <class 'str'>
BBVA+Av+arcos+De+La+Fra+Av+arcos+De+La+Frontera+Bl+6+Mzna+6 <class 'str'>
['11630 Arcos de la Frontera, Cádiz, Spain', 36.7560042, -5.8116522, 'ChIJgSwaqWehDQ0RHIzczYu8Ua0']
Larga, 57 <class 'str'>
BBVA+Larga+57 <class 'str'>
Larga, 57 - Larga, 57 <class 'str'>
BBVA+Larga+57+Larga+57 <class 'str'>
Serrana - Parque De La Serrana Bolq.5 <class 'str'>
BBVA+Serrana+Parque+De+La+Serrana+Bolq+5 <class 'str'>
['Av. la Serrana, Jerez de la Frontera, Cádiz, Spain', 36.6945246, -6.1438064, 'ChIJwYyqlqTGDQ0RYfPcHwf-lYM']
Hipercor - Av.voltaire - Ed.albatros - Local

['Av. de la Viñuela, 2, 14010 Córdoba, Spain', 37.8906701, -4.7634755, 'ChIJq6rOhYTfbA0RkBJGJ0Ar9Sk']
Arroyo Del Moro - Av. Arroyo Del Moro, 9 <class 'str'>
BBVA+Arroyo+Del+Moro+Av+Arroyo+Del+Moro+9 <class 'str'>
['Avenida Arroyo del Moro, 9, 14011 Córdoba, Spain', 37.8895076, -4.7977645, 'ChIJM3GOxETfbA0REF111oML4Es']
Trassierra - Betis, 10 <class 'str'>
BBVA+Trassierra+Betis+10 <class 'str'>
Pablo Picasso - Pablo Picasso, 35 <class 'str'>
BBVA+Pablo+Picasso+Pablo+Picasso+35 <class 'str'>
Carlos Iii - Av. Carlos Iii, 30 - 32 <class 'str'>
BBVA+Carlos+Iii+Av+Carlos+Iii+30+32 <class 'str'>
['Av. Carlos III, 68, 14100 La Carlota, Córdoba, Spain', 37.6725805, -4.9335292, 'ChIJRTpTk1IwbQ0RHr-lEt6lsmA']
Ramon Y Cajal, 2 - Derecha <class 'str'>
BBVA+Ramon+Y+Cajal+2+Derecha <class 'str'>
Corredera, 20 <class 'str'>
BBVA+Corredera+20 <class 'str'>
Carlos Iii, 18 <class 'str'>
BBVA+Carlos+Iii+18 <class 'str'>
Corredera, 1 <class 'str'>
BBVA+Corredera+1 <class 'str'>
Puerta La Muela, 9 <class 's

Av. Madrid Esq Av. Barcelona <class 'str'>
BBVA+Av+Madrid+Esq+Av+Barcelona <class 'str'>
['Av. de Madrid, 79, 08028 Barcelona, Spain', 41.3785558, 2.1269961, 'ChIJoXxpEIuYpBIR3OG0b0zm-k8']
Corrales - C.c.nuevo Corrales, Locales 2 Y 11 <class 'str'>
BBVA+Corrales+C+c+nuevo+Corrales+Locales+2+Y+11 <class 'str'>
Venida De La Virgen, 20 <class 'str'>
BBVA+Venida+De+La+Virgen+20 <class 'str'>
['Calle Venida de la Virgen, 20, 21730 Almonte, Huelva, Spain', 37.2613269, -6.5170596, 'ChIJY6ipIEf6EQ0RC5JURASo4r4']
Blas Infante, C/v Licenc.carrion L 3 <class 'str'>
BBVA+Blas+Infante+C+v+Licenc+carrion+L+3 <class 'str'>
C/ Isla Cristina, Nº 4 <class 'str'>
BBVA+C+Isla+Cristina+N+4 <class 'str'>
['Plaza de las Flores, 10, 21410 Isla Cristina, Huelva, Spain', 37.2032038, -7.3266622, 'ChIJ-YVR8vYkEA0RWlPk9eOX7gg']
Pz. Redonda, 3 <class 'str'>
BBVA+Pz+Redonda+3 <class 'str'>
Empresas - Vazquez Lopez, 7 - 1ª Planta <class 'str'>
BBVA+Empresas+Vazquez+Lopez+7+1+Planta <class 'str'>
Plus Ultra - Plus Ult

Huelin - Tomas De Echeverria, 1 <class 'str'>
BBVA+Huelin+Tomas+De+Echeverria+1 <class 'str'>
['Calle Tomás Echeverría, 1, 29002 Málaga, Spain', 36.7049957, -4.4361805, 'ChIJDdxoKoL3cg0RYGExiNrsG8U']
Av. La Paloma - Av. De La Paloma, 35 <class 'str'>
BBVA+Av+La+Paloma+Av+De+La+Paloma+35 <class 'str'>
['Av. de la Paloma, 35, 29003 Málaga, Spain', 36.70153579999999, -4.444296, 'ChIJkZsU0X33cg0Rfevzelk3uhI']
Las Piramides - Av. Del Pato, S/n Edif. Menfis <class 'str'>
BBVA+Las+Piramides+Av+Del+Pato+S+n+Edif+Menfis <class 'str'>
Los Girasoles - Av. Velazquez, 74 <class 'str'>
BBVA+Los+Girasoles+Av+Velazquez+74 <class 'str'>
Larios, 12 - Marques De Larios, 12 - 14 <class 'str'>
BBVA+Larios+12+Marques+De+Larios+12+14 <class 'str'>
Marques De Larios, 12 - 14 <class 'str'>
BBVA+Marques+De+Larios+12+14 <class 'str'>
['Calle Marqués de Larios, 12, 14, 29005 Málaga, Spain', 36.7185732, -4.4215721, 'ChIJ91ZCTJT3cg0RS9rlzaGH5Tc']
Ortega Y Gasset - Av. Jose Ortega Y Gasset, 43 <class 'str'>
BBVA+Ort

...

['Calle de Santiago Beyro, 1 - A, 38007 Santa Cruz de Tenerife, Spain', 28.46465, -16.27179, 'ChIJYYEbbIzMQQwRrKMXHzZG8-I']
La Salud - Av. Venezuela, 67 <class 'str'>
BBVA+La+Salud+Av+Venezuela+67 <class 'str'>
Ofra Las Delicias - Crta.del Rosario, 102(ofra - Delicias) <class 'str'>
BBVA+Ofra+Las+Delicias+Crta+del+Rosario+102+ofra+Delicias+ <class 'str'>
['9 Riverside Dr, East Hartford, CT 06118, USA', 41.74963, -72.64359639999999, 'ChIJWx2SP79T5okRMBu3P1PFILY']
Barranco Grande - Cr.gral.del Sur Km.7,5 - Andira - Bl1 - Ba <class 'str'>
BBVA+Barranco+Grande+Cr+gral+del+Sur+Km+7+5+Andira+Bl1+Ba <class 'str'>
['Calle Cr. Gral. Del Sur Km. 7,5, 38108 Santa Cruz de Tenerife, Spain', 28.436435, -16.3073078, 'ChIJK2WrSifNQQwR8REyvZYeROw']
España, 29 (carretera General) <class 'str'>
BBVA+Espa+a+29+carretera+General+ <class 'str'>
La Laguna - Av. Taco, 142 Esquina Calvario <class 'str'>
BBVA+La+Laguna+Av+Taco+142+Esquina+Calvario <class 'str'>
Carretera General Del Norte, 43 <class 'str'>
BBVA

['Calle del Monasterio de Urdax, 45, 31011 Pamplona, Navarra, Spain', 42.8165542, -1.6583831, 'ChIJ54jLbIuSUA0RUdHY27YQzSI']
La Rochapea - Av.marcelo Celayeta, 84 - 86 <class 'str'>
BBVA+La+Rochapea+Av+marcelo+Celayeta+84+86 <class 'str'>
['Av. Marcelo Celayeta, 84-86, 31014 Pamplona, Navarra, Spain', 42.8284522, -1.6472938, 'ChIJ_XPnOuaSUA0R6HvpO4PhRYM']
Mendillorri - Concejo De Egues, 9 <class 'str'>
BBVA+Mendillorri+Concejo+De+Egues+9 <class 'str'>
['Calle Concejo de Egüés, 9, 31016 Pamplona, Navarra, Spain', 42.8127358, -1.6124039, 'ChIJdySI752TUA0RV5dFWZM8z7E']
Irurzun, 13 <class 'str'>
BBVA+Irurzun+13 <class 'str'>
['Calle Irurzun, 13, 31350 Peralta, Navarra, Spain', 42.3391159, -1.7994561, 'ChIJ3SoTSFZiWg0RqGlrjIdZdB4']
Ctra .estella, 15 <class 'str'>
BBVA+Ctra+estella+15 <class 'str'>
['9 Riverside Dr, East Hartford, CT 06118, USA', 41.74963, -72.64359639999999, 'ChIJWx2SP79T5okRMBu3P1PFILY']
Mayor, 37 <class 'str'>
BBVA+Mayor+37 <class 'str'>
Teofano Cortes, 2 <class 'str'>
BB

Teatinos - Reyes Catolicos, 5 <class 'str'>
BBVA+Teatinos+Reyes+Catolicos+5 <class 'str'>
['Calle Reyes Católicos, 5, 33011 Oviedo, Asturias, Spain', 43.3729203, -5.834992, 'ChIJVy7defSMNg0R_NcCBPktuEY']
Corredoria - Pz. De Cuatro Caños, 8 Bajo <class 'str'>
BBVA+Corredoria+Pz+De+Cuatro+Ca+os+8+Bajo <class 'str'>
Bermudez De Castro - Bermudez De Castro, 6 <class 'str'>
BBVA+Bermudez+De+Castro+Bermudez+De+Castro+6 <class 'str'>
La Florida - Pl. Gabino Diaz Merchan, 1 <class 'str'>
BBVA+La+Florida+Pl+Gabino+Diaz+Merchan+1 <class 'str'>
['Plaza Gabino Díaz Merchán, 1, 33012 Oviedo, Asturias, Spain', 43.36840530000001, -5.867602, 'ChIJL_dlogiNNg0RMC-5Hu4R8iA']
Leon Y Escosura - Leon Y Escosura, 3 <class 'str'>
BBVA+Leon+Y+Escosura+Leon+Y+Escosura+3 <class 'str'>
['Calle León y Escosura, 3, 33013 Oviedo, Asturias, Spain', 43.3625347, -5.8609129, 'ChIJc7sGsxyNNg0R9qINAxiBQlA']
Argañosa - Alejandro Casona, 1 <class 'str'>
BBVA+Arga+osa+Alejandro+Casona+1 <class 'str'>
['Calle Alejandro Casona

['Av. Infante Juan Manuel, 10, 30011 Murcia, Spain', 37.9809102, -1.1209597, 'ChIJmRMoEg-CYw0R1HGXKB-uenk']
Espinardo - Mayor (espinardo), 105 <class 'str'>
BBVA+Espinardo+Mayor+espinardo+105 <class 'str'>
['Calle Mayor, 105, 30100 Espinardo, Murcia, Spain', 38.0107825, -1.1542422, 'ChIJ0fRpWZ-BYw0RhAEqdqxOjtY']
Mayor - Mayor, 52 <class 'str'>
BBVA+Mayor+Mayor+52 <class 'str'>
Puente Tocinos - Mayor, 110 <class 'str'>
BBVA+Puente+Tocinos+Mayor+110 <class 'str'>
['Calle Mayor, 110, 30006 Puente Tocinos, Murcia, Spain', 37.9940569, -1.1006016, 'ChIJH86AnmCCYw0Rx9ovF5g-h8w']
Av. Dr. Meca - Cr. Pto.mazarron <class 'str'>
BBVA+Av+Dr+Meca+Cr+Pto+mazarron <class 'str'>
['Av. Dr. Meca, 30860 Puerto de Mazarrón, Murcia, Spain', 37.5718574, -1.2664787, 'ChIJCdcyRhahZA0RP0wNQL9SAMU']
Ctra. Alicante - Cartagena <class 'str'>
BBVA+Ctra+Alicante+Cartagena <class 'str'>
['Calle Mayor, 27-29, Planta 1, 30201 Cartagena, Murcia, Spain', 37.6009005, -0.9867004, 'ChIJIUHmYANCYw0RMzG3JC0k3O4']
Calle Reyes 

,office_number,office_direction,municipality_name,province_name,postal_code,geo_extraction
0,92.0,"Natalio Rivas, 23",Adra,Almería,4770,
1,5764.0,Av. Carlos Iii Esq. C/lima,Aguadulce,Almería,4720,"[Av. de Carlos III, 04720 Aguadulce, Almería, ..."
2,5461.0,"Empresas - Paseo Almeria, 41",Almeria,Almería,4001,"[Paseo de Almería, 41, 04001 Almería, Spain, 3..."
3,1809.0,"Ps. Almeria 41 - Ps. Almeria, 41",Almeria,Almería,4001,"[Paseo de Almería, 41, 04001 Almería, Spain, 3..."
4,6828.0,"Edf Oliveros - Canonigo Molina Alonso, 6",Almeria,Almería,4004,"[Calle Canónigo Molina Alonso, 6, 04004 Almerí..."


In [13]:
bbva_offices.head()

,office_number,office_direction,municipality_name,province_name,postal_code,geo_extraction
0,92.0,"Natalio Rivas, 23",Adra,Almería,4770,
1,5764.0,Av. Carlos Iii Esq. C/lima,Aguadulce,Almería,4720,"[Av. de Carlos III, 04720 Aguadulce, Almería, ..."
2,5461.0,"Empresas - Paseo Almeria, 41",Almeria,Almería,4001,"[Paseo de Almería, 41, 04001 Almería, Spain, 3..."
3,1809.0,"Ps. Almeria 41 - Ps. Almeria, 41",Almeria,Almería,4001,"[Paseo de Almería, 41, 04001 Almería, Spain, 3..."
4,6828.0,"Edf Oliveros - Canonigo Molina Alonso, 6",Almeria,Almería,4004,"[Calle Canónigo Molina Alonso, 6, 04004 Almerí..."


In [14]:
bbva_offices_with_geocode_path = data_path + 'bbva_offices_with_geocode.csv'

#Export to .csv
bbva_offices.to_csv(bbva_offices_with_geocode_path, index=False)